# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
              
        for line in csvreader:
            full_data_rows_list.append(line)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
#Query Description: In this query, I used itemInSession as the partition key and sessionId as my clustering key. Each partition is uniquely identified by itemInSession while sessionId was used to uniquely identify the rows within a partition to sort the data by the value of num
query1 = "SELECT artist_name, song_title, song_length FROM song_info_by_session WHERE sessionId = 338 AND itemInSession = 4 "   

In [9]:
queryDrop1 = "DROP TABLE IF EXISTS song_info_by_session "
try:
    print("Execute drop table song_info_by_session ")
    session.execute(queryDrop1)
except Exception as e:
    print(e)

Execute drop table song_info_by_session 


In [10]:
queryCreate1 = "CREATE TABLE IF NOT EXISTS song_info_by_session "
queryCreate1 = queryCreate1 + "(artist_name text, song_title text, song_length float, itemInSession int, sessionId int, PRIMARY KEY (itemInSession, sessionId))"
try:
    print("Execute create table song_info_by_session ")
    session.execute(queryCreate1)
except Exception as e:
    print(e)

Execute create table song_info_by_session 


In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (artist_name, song_title, song_length, itemInSession, sessionId)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        song_length = float(line[5])
        itemInSession = int(line[3])
        sessionId = int(line[8])
        session.execute(query, (line[0], line[9], song_length, itemInSession, sessionId))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
#Query Description: In this query, I used set of (sessionId, userId) as the partition key and itemInSession as my clustering key. Each partition is uniquely identified by (sessionId, userId) while itemInSession was used to uniquely identify the rows within a partition to sort the data by the value of num
query2 = "SELECT artist_name, song_title, first_name, last_name FROM song_info_by_user WHERE userId = 10 AND sessionid = 182 order by itemInSession " 

                    

In [14]:
queryDrop2 = "DROP TABLE IF EXISTS song_info_by_user "
try:
    print("Execute drop table song_info_by_user ")
    session.execute(queryDrop2)
except Exception as e:
    print(e)

Execute drop table song_info_by_user 


In [15]:
queryCreate2 = "CREATE TABLE IF NOT EXISTS song_info_by_user "
queryCreate2 = queryCreate2 + "(artist_name text, song_title text, first_name text, last_name text, itemInSession int, sessionId int, userId int, PRIMARY KEY ((sessionId, userId),itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) "
try:
    print("Execute create table song_info_by_user ")
    session.execute(queryCreate2)
except Exception as e:
    print(e)

Execute create table song_info_by_user 


In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_user (artist_name, song_title, first_name, last_name, itemInSession, sessionId, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        itemInSession = int(line[3])
        sessionId = int(line[8])
        userId = int(line[10])
        session.execute(query, (line[0], line[9], line[1], line[4], itemInSession, sessionId, userId))               

In [17]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [18]:
#Query Description: In this query, I used set of song_title as the partition key and userId as my clustering key. Each partition is uniquely identified by song_title while userId was used to uniquely identify the rows within a partition to sort the data by the value of num
query3 = "SELECT first_name, last_name,song_title, itemInSession, sessionId, userId FROM song_info_by_title WHERE song_title = 'All Hands Against His Own'" 


In [19]:
queryDrop3 = "DROP TABLE IF EXISTS song_info_by_title "
try:
    print("Execute drop table song_info_by_title ")
    session.execute(queryDrop3)
except Exception as e:
    print(e)

Execute drop table song_info_by_title 


In [20]:
queryCreate3 = "CREATE TABLE IF NOT EXISTS song_info_by_title "
queryCreate3 = queryCreate3 + "(song_title text, first_name text, last_name text, itemInSession int, sessionId int, userId int, PRIMARY KEY ((song_title), userId)) "
try:
    print("Execute create table song_info_by_title ")
    session.execute(queryCreate3)
except Exception as e:
    print(e)

Execute create table song_info_by_title 


In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_title (song_title, first_name, last_name, itemInSession, sessionId, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        itemInSession = int(line[3])
        sessionId = int(line[8])
        userId = int(line[10])
        session.execute(query, (line[9], line[1], line[4], itemInSession, sessionId, userId)) 

In [22]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [24]:
query = "drop table song_info_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [25]:
query = "drop table song_info_by_title"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()